In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg #загрузка актуальной версии whisper
!pip install git+https://github.com/eternnoir/pyTelegramBotAPI.git
!pip install moviepy
!wget   http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 # DOWNLOAD LINK
!bunzip2 /content/shape_predictor_68_face_landmarks.dat.bz2
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install matplotlib>=3.3.2
BRANCH = 'r1.21.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
# Loading weights, config and example wav for CTC-model
!wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/emo_model_weights.ckpt
!wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/emo_model_config.yaml

In [ ]:
token = "token"
import cv2
import numpy as np
import dlib
import whisper #импортируется модуль whisper
model1=whisper.load_model('medium') #загружается размер модели small(можно указать любую модель Whisper(base,tiny,small, medium, large))
import telebot
import os
from moviepy.editor import *
from google.colab.patches import cv2_imshow
import nltk # Импортируем библиотеку nltk
nltk.download('stopwords') # Импортируем корпус с русскими стоп-словами
nltk.download('punkt') # Загружаем необходимые данные punkt, которые используются для токенизации текста.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
datFile =  "/content/shape_predictor_68_face_landmarks.dat"
from typing import List, Union
from omegaconf import OmegaConf
import torch
import torchaudio
import soundfile as sf
from omegaconf import DictConfig, ListConfig
import hydra

class SpecScaler(torch.nn.Module):
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return torch.log(x.clamp_(1e-9, 1e9))


class GigaAMEmo(torch.nn.Module):
    def __init__(self, conf: Union[DictConfig, ListConfig]):
        super().__init__()
        self.id2name = conf.id2name
        self.feature_extractor = hydra.utils.instantiate(conf.feature_extractor)
        self.conformer = hydra.utils.instantiate(conf.encoder)
        self.linear_head = hydra.utils.instantiate(conf.classification_head)

    def forward(self, features, features_length=None):
        if features.dim() == 2:
            features = features.unsqueeze(0)
        if not features_length:
            features_length = torch.ones(features.shape[0]) * features.shape[-1]
            features_length = features_length.to(features.device)
        encoded, _ = self.conformer(audio_signal=features, length=features_length)
        encoded_pooled = torch.nn.functional.avg_pool1d(
            encoded, kernel_size=encoded.shape[-1]
        ).squeeze(-1)

        logits = self.linear_head(encoded_pooled)
        return logits

    def get_probs(self, audio_path: str) -> List[List[float]]:
        audio_signal, _ = sf.read(audio_path, dtype="float32")
        features = self.feature_extractor(torch.tensor(audio_signal).float().to(next(self.parameters()).device))
        logits = self.forward(features)
        probs = torch.nn.functional.softmax(logits).detach().tolist()
        return probs

import locale
locale.getpreferredencoding = lambda: "UTF-8"
from omegaconf import OmegaConf
import torchaudio
from pydub import AudioSegment
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')


In [ ]:
bot = telebot.TeleBot(token)
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.reply_to(message, " Приветствую! Это бот для анализа выступления спикера."
	+" \n Пришлие видео и я пришлю его анализ."
	+" \n Анализ включает в себя:"
	    +" \n - текст, без слов паразитов"
	    +" \n - эмоцию которую транслирует спикер через текст"
	    +" \n - количество излишне больших наклонов головы"
	    +" \n - количество отвода глаз вправо и влево"
	    +" \n - сколько раз поджал губы"
	    +" \n По команде: /help или /start будет отправлено это сообщение.")
############################## video-analys
@bot.message_handler(content_types=['video'])
def send_text(message):
  bot.reply_to(message, 'Получил')
  file_info = bot.get_file(message.video.file_id)
################################ check folders
  dirs = set(os.listdir())
  if "audio" not in dirs:
    os.mkdir("/content/audio")
    dirs = set(os.listdir())
  if "videos" not in dirs:
    os.mkdir("/content/videos")
    dirs = set(os.listdir())
################################ download
  bot.reply_to(message, "Скачиваю")
  downloaded_file = bot.download_file(file_info.file_path)
  src =  file_info.file_path
  with open(src, 'wb') as new_file:
      new_file.write(downloaded_file)
################################ recive vido+audio
  bot.reply_to(message, 'Извлекаю аудио и видео')
  video = VideoFileClip(src)
  video.audio.write_audiofile("audio.mp3")
  sound = AudioSegment.from_mp3('/content/audio.mp3')
  sound = sound.set_channels(1)
  sound.export("/content/audio/audio.mp3", format="mp3")
################################ work with audio
  for elem in os.listdir("/content/audio"):
    if elem ==".ipynb_checkpoints":
      pass
    else:
      bot.reply_to(message, "Анализирую аудио")
      src = "/content/audio/"+str(elem)
      with open(src,'rb') as f:
        #print(src)
        ################ recive text
        result=model1.transcribe(src,fp16=False) #транскрибация файла. Файл необходимо загрузить в colab,раздел "файлы". Параметр fp16=False указывает на использование 32-битного числового формата при транскрибировании.
        result_text=result['text'] # Результат работы модели сохраняется в переменную result
        ################ filter text
        result_text = result_text.lower() #Перевод в нижний регистр
        result_text2 = result_text.replace('.', '').replace(',', '').replace('–', '').replace('!', '').replace('?', '') #Чистим текст от знаков пунктуации
        result_text3 = result_text2.replace("wasn't", "was not").replace("'m", "am").replace("'re", "are").replace("'s", "is").replace("'d", "would").replace("'d", "had").replace("'d", "did").replace("'ll", "will").replace("'ve", "have").replace("n't", "not").replace("can't", "cannot").replace("won't", "will not").replace("shan't", "shall not").replace("wouldn't", "would not").replace("couldn't", "could not").replace("shouldn't", "should not").replace("mustn't", "must not").replace("hasn't", "has not").replace("haven't", "have not").replace("hadn't", "had not").replace("isn't", "is not").replace("aren't", "are not").replace("weren't", "were not").replace("don't", "do not").replace("doesn't", "does not").replace("didn't", "did not").replace("ain't", "am not").replace("ain't", "is not").replace("ain't", "are not").replace("let's", "let us").replace("needs't", "needs not").replace("use't", "used not").replace("dare't", "dare not").replace("may'd", "may would").replace("may'd", "may had").replace("might'd", "might would").replace("might'd", "might had").replace("must'd", "must would").replace("must'd", "must had").replace("shall'd", "shall would").replace("shall'd", "shall had").replace("should'd", "should would").replace("should'd", "should had").replace("will'd", "will would").replace("will'd", "will had").replace("would'd", "would would").replace("would'd", "would had")
        tokens = word_tokenize(result_text3) # Разбиваем текст на слова
        stop_words = stopwords.words('russian') # Русские слова-паразиты
        stop_words += stopwords.words('english') # Английские слова-паразиты
        newStopWords = ['короче', 'алло', 'таки', 'там', 'вот', 'так', 'уже', 'такой', 'такая', 'такие', 'как', 'бы', 'то', 'есть', 'эта', 'ну', 'блин', 'вообще', 'вон', 'это']
        stop_words.extend(newStopWords) #Расширяем список русских слов-паразитов
        filtered_tokens = [word for word in tokens if word not in stop_words] # Очищаем текст от слов-паразитов
        filtered_audio = " ".join(filtered_tokens)
        bot.reply_to(message, "Отфильтрованный текст от слов-паразитов: "+filtered_audio)
        ################ recive text emotion
        model_config = 'emo_model_config.yaml'
        model_weights = 'emo_model_weights.ckpt'
        audio_path = src[:]
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        conf = OmegaConf.load(model_config)
        model = GigaAMEmo(conf)
        ckpt = torch.load(model_weights, map_location="cpu")
        model.load_state_dict(ckpt, strict=False)
        model = model.to(device)
        model.eval()
        with torch.no_grad():
            probs = model.get_probs(audio_path)[0]
        emotion = max(enumerate(probs), key=lambda item: item[1])
        bot.reply_to(message, "Эмоция выступления: "+str(model.id2name[emotion[0]]))
      ################clear
      os.remove(src)
################################ work with video
  for elem in os.listdir("/content/videos"):
    if elem ==".ipynb_checkpoints":
      pass
    else:
      src = "/content/videos/"+str(elem)
      with open(src,'rb') as f:
        ################ count head tilt
        capture = cv2.VideoCapture(filename = src)
        angle = 0
        head_tilt = False
        head_tilt_count = 0
        while True:
            ret, frame = capture.read()
            if frame is None:
              break
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.1, 5)
            x, y, w, h = 0, 0, 0, 0
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            eyes = eye_cascade.detectMultiScale(gray[y:(y + h), x:(x + w)], 1.1, 4)
            index = 0
            eye_1 = [None, None, None, None]
            eye_2 = [None, None, None, None]
            for (ex, ey, ew, eh) in eyes:
              if index == 0:
                  eye_1 = [ex, ey, ew, eh]
              elif index == 1:
                  eye_2 = [ex, ey, ew, eh]
              cv2.rectangle(frame[y:(y + h), x:(x + w)], (ex, ey),
                (ex + ew, ey + eh), (0, 0, 255), 2)
              index = index + 1
            if (eye_1[0] is not None) and (eye_2[0] is not None):
              if eye_1[0] < eye_2[0]:
                  left_eye = eye_1
                  right_eye = eye_2
              else:
                  left_eye = eye_2
                  right_eye = eye_1
              left_eye_center = (
              int(left_eye[0] + (left_eye[2] / 2)),
              int(left_eye[1] + (left_eye[3] / 2)))
              right_eye_center = (
              int(right_eye[0] + (right_eye[2] / 2)),
              int(right_eye[1] + (right_eye[3] / 2)))
              left_eye_x = left_eye_center[0]
              left_eye_y = left_eye_center[1]
              right_eye_x = right_eye_center[0]
              right_eye_y = right_eye_center[1]
              delta_x = right_eye_x - left_eye_x
              delta_y = right_eye_y - left_eye_y
              if delta_x != 0:
                angle = np.arctan(delta_y / delta_x)
              else:
                angle = 0
            angle = (angle * 180) / np.pi
            if angle > 30:
                cv2.putText(frame, 'RIGHT TILT :' + str(int(angle))+' degrees',
                            (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (0, 0, 0), 2, cv2.LINE_4)
                if not head_tilt:
                    head_tilt = True
                    head_tilt_count += 1
            elif angle < -30:
                cv2.putText(frame, 'LEFT TILT :' + str(int(angle))+' degrees',
                            (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (0, 0, 0), 2, cv2.LINE_4)
                if not head_tilt:
                    head_tilt = True
                    head_tilt_count += 1
            else:
                cv2.putText(frame, 'STRAIGHT :', (20, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (0, 0, 0), 2, cv2.LINE_4)
                head_tilt = False
            cv2_imshow(frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        msg1='Number of head tilts: '+ str(head_tilt_count)
        capture.release()
        cv2.destroyAllWindows()
        bot.reply_to(message, msg1)
      ################ count eyes moving
      capture = cv2.VideoCapture(filename = src)
      previous_position = None
      left_eye_counter = 0
      right_eye_counter = 0
      while True:
          ret, frame = capture.read()
          if frame is None:
            break
          gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)
          current_position = None
          for (ex, ey, ew, eh) in eyes:
              cv2.rectangle(frame, (ex, ey), (ex + ew, ey + eh), (255, 0, 0), 2)
              eye_center_x = ex + ew // 2
              if eye_center_x < frame.shape[1] // 2:
                  current_position = 'LEFT'
              elif eye_center_x > frame.shape[1] * 2 // 3:
                  current_position = 'RIGHT'
          if current_position != previous_position:
              if current_position == 'LEFT':
                  cv2.putText(frame, 'LEFT', (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                  left_eye_counter +=1
              elif current_position == 'RIGHT':
                  cv2.putText(frame, 'RIGHT', (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                  right_eye_counter +=1
              previous_position = current_position
          cv2_imshow(frame)
          if cv2.waitKey(1) & 0xFF == ord('q'):
              break
      msg2='Number of eye motions to right: ' +str(left_eye_counter) + ' To left: ' +str(right_eye_counter)
      bot.reply_to(message, msg2)
      capture.release()
      cv2.destroyAllWindows()
      ################ count lip presses
      detector = dlib.get_frontal_face_detector()
      predictor = dlib.shape_predictor(datFile)
      capture = cv2.VideoCapture(filename = src)
      lip_press_count = 0
      prev_lips_pressed = False
      while True:
        ret, frame = capture.read()
        if frame is None:
          print("Error: Failed to capture frame from camera.1")
          break
        if not ret:
          print("Error: Failed to capture frame from camera.2")
          break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        for face in faces:
          landmarks = predictor(gray, face)
          mouth_points = []
          for n in range(48, 68):
              x = landmarks.part(n).x
              y = landmarks.part(n).y
              mouth_points.append((x, y))

          # Calculate the distance between upper and lower lip
          lip_distance = mouth_points[15][1] - mouth_points[16][1]

          # If the distance is less than a threshold and lips were not previously pressed, increment the counter
          if lip_distance < 1 and not prev_lips_pressed:
              lip_press_count += 1
              prev_lips_pressed = True
          elif lip_distance >= 1:
              prev_lips_pressed = False
          x, y, w, h = cv2.boundingRect(np.array(mouth_points))
          cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2_imshow(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
          break
      msg3="Total lip presses: " + str(lip_press_count)
      bot.reply_to(message, msg3)
      ################ clear
      os.remove(src)
################################
bot.infinity_polling()